In [ ]:
!pip install neat-python

In [ ]:
import neat
import numpy as np

In [ ]:
class board():
  def __init__(self):
    self.state = np.array([
      [' ', ' ', ' '],
      [' ', ' ', ' '],
      [' ', ' ', ' '],
    ])

  def state_vec(self, turn):
    val = self.state.ravel()
    val = (val == 'O').astype(int) - (val == 'X').astype(int)
    val *= 1-2*(turn%2)
    return val

  def done(self):
    if ' ' not in self.state:
      return True
    v = self.victor()
    return v != 0

  def victor(self):
    for row in self.state:
      if (row == 'O').all():
        return 1
      if (row == 'X').all():
        return -1
    for col in self.state.T:
      if (col == 'O').all():
        return 1
      if (col == 'X').all():
        return -1
    return 0

  def move(self, position, symbol):
    row,col = position//3, position%3
    if self.state[row,col] != ' ':
      return False
    self.state[row,col] = symbol
    return True
  
  def valid_mask(self):
    return 1-np.abs(self.state_vec(0))

In [ ]:
def compete(net1, net2):
  players = [(net1, 'O'), (net2, 'X')]
  game_board = board()
  turn = 0
  while not game_board.done():
    #print(f"turn: {turn}")
    net, symbol = players[turn%2]
    choice_disribution = net.activate(game_board.state_vec(turn))
    choice_disribution *= game_board.valid_mask()
    choice = np.argmax(choice_disribution)
    valid = game_board.move(choice, symbol)
    #print(f"player {turn%2} chooses {choice}")
    #print(game_board.state)
    if not valid:
      return (turn%2)*2 -1
    turn += 1
  return game_board.victor()

def random_compete(net, verbose=False):
  order = np.random.randint(0,2)
  net_symbol = ['O', 'X'][order]
  vs_symbol = ['O', 'X'][1-(order%2)]
  if verbose:
    print(f"{order} ({net_symbol}) is net")
  game_board = board()
  turn = 0
  while not game_board.done():
    if turn%2 == order:
      choice_disribution = net.activate(game_board.state_vec(turn))
      #print(choice_disribution)
      choice_disribution *= game_board.valid_mask()
      #print(choice_disribution)
      choice = np.argmax(choice_disribution)
      valid = game_board.move(choice, net_symbol)
    else:
      choice_disribution = np.random.rand(game_board.state.size)
      choice_disribution *= game_board.valid_mask()
      choice = np.argmax(choice_disribution)
      valid = game_board.move(choice, vs_symbol)
    if verbose:
      print(f"turn: {turn}")
      print(f"player {turn%2} chooses {choice}")
      print(game_board.state)
    turn += 1
  result = game_board.victor()*(1-2*order)
  if verbose:
    print(result)
  return result

def eval_genomes(genomes, config):
  rand_fit = 0
  for genome_id, genome in genomes:
    genome.fitness = 0.0
    net = neat.nn.FeedForwardNetwork.create(genome, config)
    for i in range(10):
      genome.fitness += random_compete(net)
    #print(f"{genome_id} fitness: {genome.fitness}")
    rand_fit += genome.fitness
  print("total:",rand_fit)
  for i in np.random.permutation(len(genomes)):
    for j in np.random.permutation(len(genomes)):
      genome1_id, genome1 = genomes[i]
      genome2_id, genome2 = genomes[j]
      if genome1_id == genome2_id:
        continue
      #print(f"{genome1_id} vs. {genome2_id}")
      net1 = neat.nn.FeedForwardNetwork.create(genome1, config)
      net2 = neat.nn.FeedForwardNetwork.create(genome2, config)
      result = compete(net1, net2)
      #print(f"result: {result}")
      #print()
      genome1.fitness += result
      genome2.fitness -= result

In [ ]:
def run(config_file):
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 10)

    # Display the winning genome.
    #print('\nBest genome:\n{!s}'.format(winner))
    return winner,config

In [ ]:
config_path = 'neat.config'
file = open(config_path,'w')
config_data = """[NEAT]
fitness_criterion     = max
fitness_threshold     = 100
no_fitness_termination= True
pop_size              = 10
reset_on_extinction   = False

[DefaultGenome]
# node activation options
activation_default      = sigmoid
activation_mutate_rate  = 0.0
activation_options      = sigmoid

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection add/remove rates
conn_add_prob           = 0.5
conn_delete_prob        = 0.5

# connection enable options
enabled_default         = True
enabled_mutate_rate     = 0.1

feed_forward            = True
initial_connection      = full

# node add/remove rates
node_add_prob           = 0.2
node_delete_prob        = 0.2

# network parameters
num_hidden              = 0
num_inputs              = 9
num_outputs             = 9

# node response options
response_init_mean      = 1.0
response_init_stdev     = 0.1
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.0
response_mutate_rate    = 0.0
response_replace_rate   = 0.0

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 5
species_elitism      = 4

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2
"""
file.write(config_data)
file.close()

In [ ]:
winner,config = run(config_path)

In [ ]:
net = neat.nn.FeedForwardNetwork.create(winner, config)

In [ ]:
game_board = board()

In [ ]:
choice = net.activate(game_board.state_vec(0))
print(choice)
choices = np.argsort(choice)[::-1]
print(choices)
for choice in choices:
  valid = game_board.move(choice, 'O')
  print(valid, choice)
  if valid:
    break
print(game_board.state)

In [ ]:
valid = game_board.move(7, 'X')
print(valid)
print(game_board.state)

In [ ]:
rand_fit = 0
n = 1000
for i in range(n):
  rand_fit += random_compete(net)
rand_fit/n